In [1]:
import ee
import geemap
import numpy as np
import sys
sys.path.insert(1, '/Users/gopal/Projects/ml/downloadGEErasters')
import rs
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from rs import prep_tm5_ic, prep_etm7_ic, prep_oli8_ic, prep_landsat_clouds, landsat_season_summary

In [3]:
import importlib
importlib.reload(rs)

<module 'rs' from '/Users/gopal/Projects/ml/downloadGEErasters/rs.py'>

In [2]:
ee.Initialize()

In [4]:
ee_geometry_box = ee.Geometry.Polygon(
        [[[77.42201334186198, 11.398136848502652],
          [77.42201334186198, 10.873987098476817],
          [78.33387857623698, 10.873987098476817],
          [78.33387857623698, 11.398136848502652]]], None, False)

In [5]:
ee_geometry = ee.FeatureCollection("users/gopalpenny/cauvery/Cauvery_boundary5")


In [37]:
# Prepare and merge landsat data
tm5 = prep_tm5_ic(ee_geometry) 
etm7 = prep_etm7_ic(ee_geometry)
oli8 = prep_oli8_ic(ee_geometry) 
landsat = (oli8 
    .merge(tm5) 
    .merge(etm7) #.filterDate('1990-01-01', '2003-03-31'))
    .filterMetadata('CLOUD_COVER', 'less_than', 40))


In [38]:
landsat_clouds = prep_landsat_clouds(landsat)

In [40]:
def add_ndvi_band(im):
    im = im.addBands(im.normalizedDifference(['nir','red']).rename('NDVI')) 
    im = im.addBands(im.normalizedDifference(['green','swir1']).rename('MNDWI'))
    im = im.updateMask(im.select(['clouds_shadows']).eq(0))
    im = im.float()
    return im

landsat_bands = landsat_clouds.map(add_ndvi_band)
# landsat_bands.size().getInfo()


In [41]:
# landsat_bands.first().bandNames().getInfo()

In [42]:
def get_im_seasons(ic, monsoon_year, ndvi_threshold = 0.2):
    
    print('Starting monsoon year:', monsoon_year)
    
    median_bands = ['swir2','swir1','nir','red','green','blue','NDVI', 'MNDWI']
    max_bands = ['NDVI']
    count_bands = ['NDVI']
    # specify the dates for each of the seasons
    kharif_start_dt = datetime.strptime(str(monsoon_year) + "-06-01", "%Y-%m-%d")
    kharif_start_date = datetime.strftime(kharif_start_dt, "%Y-%m-%d")
    rabi_start_date = datetime.strftime(kharif_start_dt + relativedelta(months = 4), "%Y-%m-%d")
    summer_start_date = datetime.strftime(kharif_start_dt + relativedelta(months = 8), "%Y-%m-%d")
    summer_end_date = datetime.strftime(kharif_start_dt + relativedelta(months = 12), "%Y-%m-%d")

    # calculate the median bands for each of the seasons
    im_kharif = landsat_season_summary(ic, kharif_start_date, rabi_start_date, 'kharif', median_bands, max_bands, count_bands)
    im_rabi = landsat_season_summary(ic, rabi_start_date, summer_start_date, 'rabi', median_bands, max_bands, count_bands)
    im_summer = landsat_season_summary(ic, summer_start_date, summer_end_date, 'summer', median_bands, max_bands, count_bands)
    # im_rabi = season_median(ic, rabi_start_date, summer_start_date, 'rabi')
    # im_summer = season_median(ic, summer_start_date, summer_end_date, 'summer')

    trees = ee.Image('UMD/hansen/global_forest_change_2022_v1_10') # https://developers.google.com/earth-engine/datasets/catalog/UMD_hansen_global_forest_change_2022_v1_10

    gfsad30 =  ee.Image('users/gopalpenny/GFSAD30').rename('gfsad30').eq(2) # 0 - water, 1 - noncrop, 2 - crop
    imp_val = 2019 - monsoon_year
    impervious = ee.Image("Tsinghua/FROM-GLC/GAIA/v10").gte(imp_val).multiply(5).rename('impervious') #https://developers.google.com/earth-engine/datasets/catalog/Tsinghua_FROM-GLC_GAIA_v10#bands
    # World settlement footprint https://developers.google.com/earth-engine/datasets/catalog/DLR_WSF_WSF2015_v1
    # water = ee.ImageCollection('GLCF/GLS_WATER') # https://developers.google.com/earth-engine/datasets/catalog/GLCF_GLS_WATER#description
    water = (ee.ImageCollection('JRC/GSW1_4/MonthlyHistory') # https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_4_MonthlyHistory
             .filterDate(kharif_start_date, summer_end_date)
             .map(lambda im: im.select('water').eq(2).rename('water'))
             .reduce(ee.Reducer.sum()).gt(2).multiply(4).rename('water'))
    
    # combine the bands from each season to a single image
    im_seasons = im_kharif \
      .addBands(im_rabi) \
      .addBands(im_summer) \
      .addBands(water) \
      .set('monsoon_year', monsoon_year)
    
    im_seasons = im_seasons.addBands(
        im_seasons.select(['NDVI_median_kharif']).gt(ndvi_threshold).rename('kharif_crops')).addBands(
        im_seasons.select(['NDVI_median_rabi']).gt(ndvi_threshold).rename('rabi_crops')).addBands(
        im_seasons.select(['NDVI_median_summer']).gt(ndvi_threshold).rename('summer_crops')) #.addBands(
        # im_seasons.expression('(im.NDVI_kharif > 0.25) + (im.NDVI_rabi > 0.25) + (im.NDVI_summer > 0.25)', 
        #   {'im':im_seasons}).rename('crops'))
    im_seasons = im_seasons.addBands(
        im_seasons.select(['kharif_crops','rabi_crops','summer_crops']).reduce(ee.Reducer.sum()).rename('crops'))
        

    return im_seasons

In [43]:
# monsoon_year = 2015
# im_seasons_05 = get_im_seasons(landsat_bands, 2005)
# im_seasons_06 = get_im_seasons(landsat_bands, 2006)
# im_seasons_06.bandNames().getInfo()

In [44]:

# im_seasons_05_25 = get_im_seasons(landsat_bands, 2005, 0.25)
# im_seasons_06_25 = get_im_seasons(landsat_bands, 2006, 0.25)
# im_seasons_07_25 = get_im_seasons(landsat_bands, 2007, 0.25)

im_seasons_00_22 = get_im_seasons(landsat_bands, 2000, 0.22)
# im_seasons_01_22 = get_im_seasons(landsat_bands, 2001, 0.22)
# im_seasons_02_22 = get_im_seasons(landsat_bands, 2002, 0.22)
# im_seasons_03_22 = get_im_seasons(landsat_bands, 2003, 0.22)
# im_seasons_04_22 = get_im_seasons(landsat_bands, 2004, 0.22)
# im_seasons_05_22 = get_im_seasons(landsat_bands, 2005, 0.22)
# im_seasons_06_22 = get_im_seasons(landsat_bands, 2006, 0.22)
# im_seasons_07_22 = get_im_seasons(landsat_bands, 2007, 0.22)
# im_seasons_08_22 = get_im_seasons(landsat_bands, 2008, 0.22)
# im_seasons_09_22 = get_im_seasons(landsat_bands, 2009, 0.22)
# im_seasons_10_22 = get_im_seasons(landsat_bands, 2010, 0.22)
# im_seasons_11_22 = get_im_seasons(landsat_bands, 2011, 0.22)
# im_seasons_12_22 = get_im_seasons(landsat_bands, 2012, 0.22)
# im_seasons_13_22 = get_im_seasons(landsat_bands, 2013, 0.22)
# im_seasons_14_22 = get_im_seasons(landsat_bands, 2014, 0.22)
im_seasons_15_22 = get_im_seasons(landsat_bands, 2015, 0.22)
im_seasons_16_22 = get_im_seasons(landsat_bands, 2016, 0.22)
im_seasons_17_22 = get_im_seasons(landsat_bands, 2017, 0.22)
im_seasons_18_22 = get_im_seasons(landsat_bands, 2018, 0.22)
im_seasons_19_22 = get_im_seasons(landsat_bands, 2019, 0.22)
# im_seasons_20_22 = get_im_seasons(landsat_bands, 2020, 0.22)
# im_seasons_21_22 = get_im_seasons(landsat_bands, 2021, 0.22)
# im_seasons_22_22 = get_im_seasons(landsat_bands, 2022, 0.22)

# im_seasons_05_20 = get_im_seasons(landsat_bands, 2005, 0.2)
# im_seasons_06_20 = get_im_seasons(landsat_bands, 2006, 0.2)
# im_seasons_07_20 = get_im_seasons(landsat_bands, 2007, 0.2)


# print(im_season)
# im_seasons_15 = get_im_seasons(landsat_bands, 2015)
# im_seasons_16 = get_im_seasons(landsat_bands, 2016)
# im_seasons_17 = get_im_seasons(landsat_bands, 2017)
# im_seasons_18 = get_im_seasons(landsat_bands, 2018)
# im_seasons_19 = get_im_`seasons(landsat_bands, 2019)
# im_seasons_20 = get_im_seasons(landsat_bands, 2020)
# im_seasons_21 = get_im_seasons(landsat_bands, 2021)

Starting monsoon year: 2000
kharif num images: 12
rabi num images: 28
summer num images: 34
Starting monsoon year: 2015
kharif num images: 42
rabi num images: 77
summer num images: 121
Starting monsoon year: 2016
kharif num images: 33
rabi num images: 93
summer num images: 111
Starting monsoon year: 2017
kharif num images: 33
rabi num images: 100
summer num images: 86
Starting monsoon year: 2018
kharif num images: 34
rabi num images: 94
summer num images: 116
Starting monsoon year: 2019
kharif num images: 36
rabi num images: 83
summer num images: 102


In [47]:
im_seasons_15_22

In [30]:
im_seasons_all = ee.ImageCollection([im_seasons_00_22,
                                    #  im_seasons_06_22,
                                    # im_seasons_07_22,
                                    # im_seasons_08_22,
                                    # im_seasons_09_22,
                                    # im_seasons_10_22,
                                    # im_seasons_11_22,
                                    # im_seasons_12_22,
                                    # im_seasons_13_22,
                                    # im_seasons_14_22,
                                    im_seasons_15_22,
                                    im_seasons_16_22,
                                    im_seasons_17_22,
                                    im_seasons_18_22,
                                    im_seasons_19_22,
                                    im_seasons_20_22,
                                    im_seasons_21_22,
                                    im_seasons_22_22])
                                    #  im_seasons_15, 
                                    #  im_seasons_16,
                                    #  im_seasons_17,
                                    #  im_seasons_18,
                                    #  im_seasons_19,
                                    #  im_seasons_20,
                                    #  im_seasons_21])


In [16]:

# m16.addLayer(im_seasons_06_22.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2006 NDVI>0.22')

In [24]:
# def get_crops_bit_adj(im):
#     monsoon_year = im.get('monsoon_year')
#     crops_year_diff = ee.Number(monsoon_year).subtract(2015)
#     crops_mult = ee.Number(4).pow(crops_year_diff)
    
#     return im.select(['crops']).multiply(crops_mult)
#     # crops_bit = 
    
    
# crops_2bit_base2015 = im_seasons_all \
#   .filter(ee.Filter.lte('monsoon_year',2015)) \
#   .map(get_crops_bit_adj) \
#   .reduce(ee.Reducer.sum())
    

In [13]:
# def select_crops_year(im):
#     monsoon_year = ee.String(im.get('monsoon_year'))
#     band_name = ee.String('y').cat(monsoon_year)
    
#     return im.select(['crops'],[band_name])
    

# crops_bands = im_seasons_all.select(['crops']).toBands()

# crops_bands.getInfo()

In [14]:
# mall = geemap.Map()

# mall.addLayer(crops_2bit_base2015,{'min':0,'max':16000},'crops_2bit_base2015')
# mall.centerObject(ee_geometry_box, 10)
# mall.addLayerControl()
# mall

In [16]:
crops_task = ee.batch.Export.image.toDrive(
    image = im_seasons_15_22,
    description = 'crops_bands',
    folder = 'classy_downloads',
    fileNamePrefix = 'karur_crops_2015_2021',
    region = ee_geometry_box,
    scale = 30)

# crops_task.start()

In [46]:
# gfsad30 = ee.Image("users/gopalpenny/GFSAD30")
m = geemap.Map()

colors = ['#D2B48C',  '#EEC900', '#90EE90',  '#228B22']
colors7 = ["#F1F1F1","#E8DCCA","#B5C8A2","#68B094","#008E9A","#395E96","#6B0077"]

m.addLayer(im_seasons_00_22.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2000')
# m.addLayer(im_seasons_06_22.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2006')
# m.addLayer(im_seasons_07_22.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2007')
m.addLayer(im_seasons_15_22.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2015')
m.addLayer(im_seasons_16_22.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2016')
m.addLayer(im_seasons_17_22.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2017')
m.addLayer(im_seasons_18_22.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2018')
m.addLayer(im_seasons_19_22.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2019')
# m.addLayer(im_seasons_20_22.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2020')
# m.addLayer(im_seasons_21_22.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2021')
# m.addLayer(im_seasons_22_22.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2022')



# m.addLayer(im_seasons_06.clip(ee_geometry), {'bands':['kharif_count'], 'min':0, 'max':6, 'palette': colors7},'kharif count 2006')
# m.addLayer(im_seasons_06.clip(ee_geometry), {'bands':['rabi_count'], 'min':0, 'max':6, 'palette': colors7},'rabi count 2006')
# m.addLayer(im_seasons_06.clip(ee_geometry), {'bands':['summer_count'], 'min':0, 'max':6, 'palette': colors7},'summer count 2006')
# m.addLayer(im_seasons_15.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2015')
# m.addLayer(im_seasons_16.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2016')
# m.addLayer(im_seasons_21.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2021')
# m.centerObject(ee_geometry, 13)
m.addLayerControl()
m.centerObject(ee_geometry,8)
m

Map(center=[11.748174793282523, 77.4497503999458], controls=(WidgetControl(options=['position', 'transparent_b…

In [52]:
m16 = geemap.Map()

colors = ['#D2B48C',  '#EEC900', '#90EE90',  '#228B22']

# m.addLayer(gfsad30, {'min':0, 'max':2},'gfsad30')
# m16.addLayer(im_seasons_16, {'bands':['SR_B5_kharif','SR_B4_kharif','SR_B3_kharif'], 'min':5000, 'max':30000},'oli8 kharif')
# m16.addLayer(im_seasons_16, {'bands':['SR_B5_rabi','SR_B4_rabi','SR_B3_rabi'], 'min':5000, 'max':30000},'oli8 rabi')

# 2000
m16.addLayer(im_seasons_00_22.clip(ee_geometry), {'bands':['nir_median_rabi','red_median_rabi','green_median_rabi'], 'min':0, 'max':30000},'rabi 2000')
m16.addLayer(im_seasons_00_22.clip(ee_geometry), {'bands':['nir_median_kharif','red_median_kharif','green_median_kharif'], 'min':0, 'max':30000},'kharif 2000')
m16.addLayer(im_seasons_00_22.clip(ee_geometry), {'bands':['nir_median_summer','red_median_summer','green_median_summer'], 'min':0, 'max':30000},'summer 2000')
m16.addLayer(im_seasons_00_22.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2000 NDVI>0.22')

# 2015
m16.addLayer(im_seasons_15_22.clip(ee_geometry), {'bands':['nir_median_kharif','red_median_kharif','green_median_kharif'], 'min':0, 'max':30000},'kharif 2015')
m16.addLayer(im_seasons_15_22.clip(ee_geometry), {'bands':['nir_median_rabi','red_median_rabi','green_median_rabi'], 'min':0, 'max':30000},'rabi 2015')
m16.addLayer(im_seasons_15_22.clip(ee_geometry), {'bands':['nir_median_summer','red_median_summer','green_median_summer'], 'min':0, 'max':30000},'summer 2015')
m16.addLayer(im_seasons_15_22.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2015 NDVI>0.22')

# 2016
m16.addLayer(im_seasons_16_22.clip(ee_geometry), {'bands':['nir_median_kharif','red_median_kharif','green_median_kharif'], 'min':0, 'max':30000},'kharif 2016')
m16.addLayer(im_seasons_16_22.clip(ee_geometry), {'bands':['nir_median_rabi','red_median_rabi','green_median_rabi'], 'min':0, 'max':30000},'rabi 2016')
m16.addLayer(im_seasons_16_22.clip(ee_geometry), {'bands':['nir_median_summer','red_median_summer','green_median_summer'], 'min':0, 'max':30000},'summer 2016')
m16.addLayer(im_seasons_16_22.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2016 NDVI>0.22')

m16.addLayer(im_seasons_18_22.clip(ee_geometry), {'bands':['nir_median_kharif','red_median_kharif','green_median_kharif'], 'min':0, 'max':30000},'kharif 2016')
m16.addLayer(im_seasons_18_22.clip(ee_geometry), {'bands':['nir_median_rabi','red_median_rabi','green_median_rabi'], 'min':0, 'max':30000},'rabi 2016')
m16.addLayer(im_seasons_18_22.clip(ee_geometry), {'bands':['nir_median_summer','red_median_summer','green_median_summer'], 'min':0, 'max':30000},'summer 2016')
m16.addLayer(im_seasons_18_22.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2016 NDVI>0.22')

# m16.addLayer(im_seasons_06_25.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2006 NDVI>0.25')
# m16.addLayer(im_seasons_15_22.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2006 NDVI>0.22')
# m16.addLayer(im_seasons_15_22.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3, 'palette': colors},'crops 2006 NDVI>0.2')
# m16.addLayer(im_seasons_06.clip(ee_geometry), {'bands':['NDVI_rabi'], 'min':0, 'max':0.5},'ndvi rabi')
# m16.addLayer(im_seasons_06.clip(ee_geometry), {'bands':['NDVI_summer'], 'min':0, 'max':0.5},'ndvi summer')
# m16.addLayer(im_seasons_06.clip(ee_geometry), {'bands':['NDVI_kharif'], 'min':0, 'max':0.5},'ndvi kharif')
# m16.addLayer(im_seasons_06.clip(ee_geometry), {'bands':['NDVI_rabi'], 'min':0, 'max':0.5},'ndvi rabi')
# m16.addLayer(im_seasons_06.clip(ee_geometry), {'bands':['NDVI_summer'], 'min':0, 'max':0.5},'ndvi summer')
# m16.addLayer(im_seasons_06.clip(ee_geometry), {'bands':['NDVI_kharif','NDVI_rabi','NDVI_summer'], 'min':0, 'max':0.5},'ndvi all')

# m16.addLayer(im_seasons_16.gt(0.25), {'bands':['NDVI_rabi'], 'min':0, 'max':0.5},'crop rabi')
# m16.addLayer(im_seasons_15, {'bands':['crops'], 'min':0, 'max':3},'crops 2015')
# m16.addLayer(im_seasons_16, {'bands':['crops'], 'min':0, 'max':3},'crops 2016')
# m16.addLayer(im_seasons_16, {'bands':['crops'], 'min':0, 'max':3},'crops 2016')
# m16.addLayer(im_seasons_16, {'bands':['MNDWI_kharif','NDVI_rabi','NDVI_summer'], 'min':0.1, 'max':0.3},'mndwi kharif')
# m16.addLayer(im_seasons_16, {'bands':['MNDWI_kharif'], 'min':-0.2, 'max':0.1},'mndwi kharif')
# m16.addLayer(im_seasons_16, {'bands':['MNDWI_rabi'], 'min':-0.2, 'max':0.1},'mndwi rabi')
# m.addLayer(oli8_summer, {'bands':['NDVI'], 'min':0, 'max':0.5},'ndvi summer')
# m.addLayer(oli8_kharif, {'bands':['MNDWI'], 'min':-0.3, 'max':0},'ndwi_median')
# m.addLayer(ee_geometry)
m16.centerObject(ee_geometry, 8)
m16.addLayerControl()
m16

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…